### Parcing vacancies from [hh.ru](https://hh.ru/?hhtmFrom=vacancy) by keywords
___

In [1]:
import pandas as pd
import requests
import re
from time import sleep


pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [2]:
# city number
area = 1

# interesting competencies to search for
search_keywords = ['sql',
                   'postgresql',
                   'pandas',
                   'power bi',
                   'power query',
                   'power pivot'
                   'dax']

search_keywords = '+or+'.join([i.replace(' ', '+') for i in search_keywords])

In [3]:
n = 10    # number of pages
columns = ['name', 'salary', 'address', 'published_at', 'alternate_url', 'employer', 'snippet', 'schedule']    # keys from json

lst = []
for page in range(n):
    url = f'http://api.hh.ru/vacancies?text={search_keywords}&area={area}&per_page=100&page={page}'
    data = requests.get(url).json()
    
    sleep(2)

    for i in data['items']:
        lst.append(i)

In [4]:
df = pd.DataFrame(lst, columns=columns)

In [5]:
df['employer'] = df['employer'].map(lambda x: x['name'])
df['schedule'] = df['schedule'].map(lambda x: x['name'])

address = pd.json_normalize(df['address'])[['metro.station_name', 'metro.line_name']]
salary = pd.json_normalize(df['salary']).iloc[:, [0, 1, 3]].rename(columns={'from':'salary_from', 'to':'salary_to'})
salary['gross'] = salary['gross'] * 1
snippet = pd.json_normalize(df['snippet'])

In [6]:
df = pd.concat([df.drop(columns=(['address', 'snippet', 'salary'])), address, snippet, salary], axis=1)

In [7]:
df['requirement'] = df['requirement'].map(lambda x: re.sub('<[^>]*>', '', x) if x != None else None)
df['responsibility'] = df['responsibility'].map(lambda x: re.sub('<[^>]*>', '', x) if x != None else None)

In [8]:
df['published_at'] = pd.to_datetime(df['published_at']).dt.tz_localize(None)
df.sort_values(by='published_at', ascending=False, inplace=True)

In [9]:
df = df[['published_at', 'employer', 'name', 'requirement', 
         'responsibility', 'salary_from', 'salary_to', 'gross', 
         'schedule', 'metro.station_name', 'metro.line_name', 'alternate_url']]

In [10]:
# adding columns for filtering

df['has_sql'] = df['requirement'].map(lambda x: 'sql' in x.lower() if x != None else None) * 1
df['has_postgresql'] = df['requirement'].map(lambda x: 'postgresql' in x.lower() if x != None else None) * 1
df['has_pandas'] = df['requirement'].map(lambda x: 'pandas' in x.lower() if x != None else None) * 1
df['has_powerbi'] = df['requirement'].map(lambda x: 'power bi' in x.lower() if x != None else None) * 1
df['has_excel'] = df['requirement'].map(lambda x: any([i in x.lower() for i in ['power query, power pivot', 'dax']]) if x != None else None) * 1

In [11]:
df.head(10)

,published_at,employer,name,requirement,responsibility,salary_from,salary_to,gross,schedule,metro.station_name,metro.line_name,alternate_url,has_sql,has_postgresql,has_pandas,has_powerbi,has_excel
384,2023-03-03 22:24:17,"ТрансМашХолдинг, Группа компаний",Экономист (себестоимость),Опыт договорной работы (как преимущество). Опы...,"Знание MS Excel (Продвинутый уровень), Power Q...",NaN,NaN,NaN,Полный день,NaN,NaN,https://hh.ru/vacancy/77037203,0,0,0,0,0
705,2023-03-03 22:20:47,Тинькофф,Риск-аналитик (кредитные риски),Высшее техническое образование. Знание SQL и P...,Построение и валидация скоринговых моделей. Оц...,NaN,NaN,NaN,Полный день,Водный стадион,Замоскворецкая,https://hh.ru/vacancy/72790924,1,0,0,0,0
938,2023-03-03 22:20:47,Тинькофф,Риск-аналитик,"Уверенное знание SQL, Python. Хорошие знания м...",Генерить и проверять гипотезы для усиления вну...,NaN,NaN,NaN,Полный день,Водный стадион,Замоскворецкая,https://hh.ru/vacancy/72790429,1,0,0,0,1
571,2023-03-03 22:17:00,Paper Planes,Консультант отдела стратегии / бизнес-консультант,"Понимание принципов корреляционного, регрессио...","Подготовка презентаций, отчетов, а также марке...",100000.0,NaN,0,Полный день,Белорусская,Кольцевая,https://hh.ru/vacancy/77181833,0,0,0,1,0
352,2023-03-03 22:08:31,Ельмикеев Евгений Иванович,Junior / Junior+php программист (стажёр),PHP 8+. Laravel 8. PostgreSQL/MySQL. - Знать к...,Основной стэк: - Работа с API. ⁃ Написанием мн...,25000.0,NaN,0,Удаленная работа,NaN,NaN,https://hh.ru/vacancy/77097272,1,1,0,0,1
0,2023-03-03 21:56:53,Ельмикеев Евгений Иванович,Аналитик Power BI (стажер),"Кстати, Power Bi работает, все хорошо. -у тебя...",Вы точно прокачаетесь в задачах и решениях. А ...,25000.0,NaN,0,Удаленная работа,NaN,NaN,https://hh.ru/vacancy/77097183,0,0,0,1,0
468,2023-03-03 21:55:11,Ельмикеев Евгений Иванович,Бизнес-аналитик (стажёр),Главная задача должности бизнес-аналитика: най...,Ведение проектов по внедрению отчетности BI в ...,25000.0,NaN,0,Удаленная работа,NaN,NaN,https://hh.ru/vacancy/77097174,0,0,0,0,0
34,2023-03-03 21:54:08,Научно-производственный центр информационных р...,Fullstack программист-разработчик WEB-приложен...,"Знание JavaScript, SQL. Опыт работы c PоstgreS...",Разработка программного обеспечения на веб-тех...,65000.0,140000.0,1,Полный день,NaN,NaN,https://hh.ru/vacancy/76612689,1,0,0,0,1
902,2023-03-03 21:38:11,ИТ-Экспертиза,Руководитель инженерного отдела,Отличное знание современных серверных технолог...,Компания «ИТ-Экспертиза» — это команда професс...,NaN,NaN,NaN,Полный день,NaN,NaN,https://hh.ru/vacancy/77181563,0,0,0,0,0
30,2023-03-03 21:20:45,МГТС,Системный аналитик со знанием MS SQL,"Знание языков программирования: Python, Sql, D...","1. Выгрузка из баз данных, анализ данных. 2. Н...",NaN,NaN,NaN,Полный день,Пролетарская,Таганско-Краснопресненская,https://hh.ru/vacancy/76536031,1,0,1,0,1


In [12]:
with pd.ExcelWriter(f'./hh_vacancy_parcer_{pd.Timestamp.today().strftime("%Y-%m-%d")}.xlsx', engine='xlsxwriter') as w:
    df.to_excel(w, sheet_name='data', index=False)

    w.sheets['data'].autofilter(0, 0, df.shape[0], df.shape[1]-1)
    w.sheets['data'].set_column(0, df.shape[1], 25)